In [1]:
import json
import re
import os
from graphviz import Graph


### Listing the top 10 files in the directory tables

In [11]:
dirpath = os.path.join( os.getcwd() , "tables")
# files = os.listdir(path=dirpath)[:]

# schema_table= []
# for file in files:
#     schema_table.append ((file.split(".")[0],file.split(".")[1]))
# schema_table.sort(key=lambda x: x[0])
    

# for item in schema_table:
#     print(item)



In [12]:
# def getParentNode(filepath):
#     parentNode = ".".join(os.path.basename(filepath).split(".")[:-1])
#     return parentNode

# def getTheFromClauseText(filepath):
#     from_text = ""
#     try:
#         fileObj = open(filepath)
#         config_dict = json.load(fileObj)
#         try:
#             # Fetch the from clause
#             from_text = config_dict["query"]["L"][0]["M"]["from"]["S"] # For query with list
#             return from_text
#         except:
#             try:
#                 from_text = config_dict["query"]["M"]["from"]["S"] # Single query without list
#                 return from_text
#             except:
#                 # Do nothing if this fails too, and return from_text as None
#                 return from_text 
#         fileObj.close()
#     except FileNotFoundError:
#         return None
#     except:
#         return None

# def getTheFirstSource(from_text):
#     if from_text:
#         p = re.compile(r"""
#             (?P<starting>\w+[.]\w+) # Get the starting text    
#             """, re.VERBOSE)

#         m = p.search(from_text)
#         starting_source = m.group('starting')
#         return starting_source

# def getTheJoinTablesList(from_text):
#     joining_source =[]
#     if from_text:
#         p = re.compile(r"""
#         join\s+(?P<table>\w+[.]\w+)
#         """, re.VERBOSE)
#         joining_source = p.findall(from_text) 
        
#         return joining_source
#     else:
#         return joining_source
    
            
# def getTableList(from_text):
#     table_list = []
#     starting_source = getTheFirstSource(from_text)
#     joining_source = getTheJoinTablesList(from_text)
    
    
#     if starting_source:
#         table_list.append(starting_source)
#         table_list = table_list + joining_source
#     return table_list


# def consolidateTableList(filepath):
#     tables_list = []
#     from_text = getTheFromClauseText(filepath)
#     table_list = getTableList(from_text)

#     return table_list

# def createBranch(filepath):
#     parentNode = getParentNode(filepath)
#     table_list = list(dict.fromkeys(consolidateTableList(filepath))) #Prevent duplicates like self join or inner join
#     tree = {}
#     if not table_list:
#         tree[parentNode] = []
#     else:
#         tree[parentNode] = table_list  
#     return tree

# def fileExists(filename):
#     filepath = os.path.join(dirpath,filename+".json")
#     return os.path.isfile(filepath)

# def getFilePath(filename):
#     filepath = os.path.join(dirpath,filename+".json")
#     return filepath
            



In [13]:
filepath = os.path.join(dirpath,'report.mlb_orgs.json')
# filepath = os.path.join(dirpath,'scout.teams.json')
# filepath = os.path.join(dirpath,'crosscheck.tags.json')
filepath = os.path.join(dirpath,'report.delivered_exceptions_hist.json')

# branches = createBranch(filepath)
# print(branches)


In [14]:
grandTreeList = []

def constructTreeList(filepath):
    branches = createBranch(filepath)
    
    for key,tables in branches.items():
        for table in tables:
            constructTreeList(getFilePath(table))
    grandTreeList.append(branches)
constructTreeList(filepath)



for item in grandTreeList:
    print(item)


        

{'broadcast.delivered_games': []}
{'report.delivered_new_pk': ['broadcast.delivered_games']}
{'base.games': []}
{'rundown.access_rules_end': []}
{'broadcast.priority_lists': []}
{'report.exp_access_rules_hist': ['rundown.access_rules_end', 'broadcast.priority_lists']}
{'report.delivered_exceptions_hist': ['report.delivered_new_pk', 'base.games', 'report.exp_access_rules_hist']}


In [7]:
class ConstructGrantTree:
    def __init__(self,grandTreeList):
        self.treeDict = {}
        self.branches = []
        self.leaves = [] 
        self.allNodes = []
        self.treeList = grandTreeList
        self.parent_child_list=[]
        self.rootNode = None
        self.removeLeafNodes()
        self.getParentChildList()       
     
    def getParentChildList(self):
        for node in self.allNodes:
            self.parent_child_list.append(self.getMyParent(node))
        self.getRootNode()
        
        for parent,item in self.parent_child_list:
            print(parent,item)
                    
    def getRootNode(self):
        for parent,item in self.parent_child_list:
            if parent == None:
                self.rootNode = item
    
    def removeLeafNodes(self):
        for dictionary in self.treeList:
            for key,values in dictionary.items():
                if values:
                    self.branches.append(key)
                    self.allNodes.append(key)
                else :
                    self.leaves.append(key)
                    self.allNodes.append(key)
                    
    def getMyParent(self,node):
        parent = None
        for dictionary in self.treeList:
            for key,values in dictionary.items():
                if node in values:
                    parent = key
        return (parent,node)
    
tree = ConstructGrantTree(grandTreeList)
print(tree.parent_child_list)


report.delivered_new_pk broadcast.delivered_games
report.delivered_exceptions_hist report.delivered_new_pk
report.delivered_exceptions_hist base.games
report.exp_access_rules_hist rundown.access_rules_end
report.exp_access_rules_hist broadcast.priority_lists
report.delivered_exceptions_hist report.exp_access_rules_hist
None report.delivered_exceptions_hist
[('report.delivered_new_pk', 'broadcast.delivered_games'), ('report.delivered_exceptions_hist', 'report.delivered_new_pk'), ('report.delivered_exceptions_hist', 'base.games'), ('report.exp_access_rules_hist', 'rundown.access_rules_end'), ('report.exp_access_rules_hist', 'broadcast.priority_lists'), ('report.delivered_exceptions_hist', 'report.exp_access_rules_hist'), (None, 'report.delivered_exceptions_hist')]


In [8]:
compound = (
    ('report.delivered_new_pk', 'broadcast.delivered_games'),
    ('report.delivered_exceptions_hist', 'report.delivered_new_pk'),
    ('report.delivered_exceptions_hist', 'base.games'),
    ('report.exp_access_rules_hist', 'rundown.access_rules_end'),
    ('report.exp_access_rules_hist', 'broadcast.priority_lists'),
    ('report.delivered_exceptions_hist', 'report.exp_access_rules_hist'),
    (None, 'report.delivered_exceptions_hist'),
    )
allNodes = ['broadcast.delivered_games', 'report.delivered_new_pk', 'base.games', 'rundown.access_rules_end', 'broadcast.priority_lists', 'report.exp_access_rules_hist', 'report.delivered_exceptions_hist']


In [9]:
compound

(('report.delivered_new_pk', 'broadcast.delivered_games'),
 ('report.delivered_exceptions_hist', 'report.delivered_new_pk'),
 ('report.delivered_exceptions_hist', 'base.games'),
 ('report.exp_access_rules_hist', 'rundown.access_rules_end'),
 ('report.exp_access_rules_hist', 'broadcast.priority_lists'),
 ('report.delivered_exceptions_hist', 'report.exp_access_rules_hist'),
 (None, 'report.delivered_exceptions_hist'))

In [10]:
rootDict = {}
rootNode = 'report.delivered_exceptions_hist'
getAllMyChildren(currentNode):


SyntaxError: invalid syntax (<ipython-input-10-ffdcca2327b4>, line 3)

In [ ]:
allNodes